# CH 6 - TP Exploiter les données en grand nombre  

Il est très facile de trouver sur Internet des données partagées en grand nombre (Opendata).
Ces données sont très souvent stockées dans des fichiers `.csv` (comma-separated values).

Le but de ce TP est d'exploiter ces données, qui sont reprises dans Python sous la forme de tableaux de dictionnaires partageant les mêmes clés.

## PARTIE 1 : lecture d'un fichier .csv

Observez le code ci-dessous. La fonction `open` permet d'ouvrir un fichier. 

`for index,ligne in enumerate(csv_file):` permet de parcourir une à une les lignes du fichier en conservant dans `index` le numéro de ligne et dans `ligne` le contenu.

Les lignes sont ensuite "découpées" sous la forme de listes grâce à la très précieuse instruction `ligne.strip().split(',')` qui prend en compte que le séparateur est la virgule.


In [ ]:
def lire_fichier_csv(path):
    '''str -> list
       retourne une liste de dictionnaires partageant les mêmes descripteurs
       issus du fichier csv dont le chemin est donné par path
       le fichier csv doit contenir une première ligne avec les descripteurs
    '''
    csv_file = open(path, "r") 
    liste = [] # creation de la liste  
    for index,ligne in enumerate(csv_file):
        if index == 0:
            # récuperation de la liste des descripteurs sur la premiere ligne
            descripteurs = ligne.strip().split(',')
        else:
            # nouvel enregistrement
            enregistrement={}
            donnees = ligne.strip().split(',')     
            for descripteur,donnee in zip(descripteurs,donnees):
                enregistrement[descripteur]=donnee
            # l'enregistrement est ajouté à la liste
            liste.append(enregistrement)    
    csv_file.close()
    return liste

### Exemple :  Les passagers du Titanic

Nous allons utiliser la fonction ci-dessus pour exploiter les données sur les passagers du Titanic stockées dans le fichier `titanic.csv`.
Nous allons :
- Constater le nombre d'enregistrements
- consulter les informations concernant le premier passager de la liste
- obtenir le nombre de survivants par un parcours de la liste 

In [ ]:
# 1 chargement du fichier
maliste = lire_fichier_csv('./titanic.csv')

In [ ]:
# Combien d'enregistements ?
print(len(maliste))

In [ ]:
# le premier de la liste (observez qu'on obtient un enregistrement)
print(maliste[0])

In [ ]:
# Combien de survuvants 
nbsurvivants=0
for passager in maliste: #passager est un dictionnaire
    if passager['Survived']=='1':
        nbsurvivants+=1
print(f"on recense {nbsurvivants} survivants.")

## EXERCICE 1 : Les communes de France  

Nous disposons d'un fichier `communes.csv` qui recense les quelques 36000 communes de France. 
En utilisant le modus operandi de l'exemple précédent, répondre aux questions suivantes.

### 1.1. Lister les descripteurs de ce fichier

### 1.2 Combien de communes sont recensées dans ce tableau ?

### 1.3 Quelle est la commune la plus haute de France ?

### 1.4 Si on considère qu'un village fait moins de 2000 habitants. Combien y a-t-il de villages en France ? 

## PARTIE 2 - Récupérer des données sur Internet

Il s'agit là aussi de récupérer un fichier `.csv`, mais il se situe sur internet (via une url). 
Le gros avantage ici, c'est qu'on peut accéder à des données actualisées en temps réel.

On utilise le module `request` qui permet d'effectuer une requète `http` sur une url via la commande :
` csvfile = requests.get(url)`. Une fois les données récupérées et converties de binaire en texte, c'est le même traitement que précédemment.

In [ ]:
def telecharger_csv(url):
    '''str -> list
       retourne une liste de dictionnaires partageant les mêmes descripteurs
       issus du fichier csv dont l'url est donné par url
       le fichier csv doit contenir une première ligne avec les descripteurs
    '''
    import requests
    csvfile = requests.get(url)
    csvfile=csvfile.text #les données sont binaires, elles sont converties en str
    lignes = csvfile.split('\n') #création d'une liste de lignes
    descripteurs=lignes[0].split(',') #liste des descripteurs
    liste=[]
    for ligne in lignes[1:-1]: #premiere et derniere ligne enlevees
        donnees=str(ligne).strip().split(',')
        enregistrement={} 
        for descripteur,donnee in zip(descripteurs,donnees): #parcours simultané de 2 listes
            enregistrement[descripteur]=donnee #données dans le dictionnaire
        liste.append(enregistrement) #on ajoute notre enregistrement à la liste
    return liste


 ## Exercice 2
 
 Nous allons télécharger la liste des tremblements de Terre recensés ces derniers mois dans le monde.

In [ ]:
maliste=telecharger_csv('http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/4.5_month.csv')

### 2.1 Lister les descripteurs de ce fichier

### 2.2 Soit  m une magnétude (l'échelle de Richter va jusqu'à 9).
Etablir la liste des tremblements de terre de magnétude >n. 
On affichera un résumé des informations (lieu, quel jour ? ...)
Effectuer plusieurs essais avec m=8 ou 7 ...

## PARTIE 3 : Création d'une carte avec folium 

La bibliothèque folium permet de créer des cartes openstreetmap. Elle n'est pas en natif dans Anaconda, il faut l'installer une fois pour toutes pour chaque ordinateur:

 à n'exécuter qu'une fois par ordinateur sur le prompt Anaconda
 
`set 'https_proxy' = https://10.0.0.1:3128` # proxy du Lycée Simone Weil

`pip install folium`

In [2]:
# importation du module Folium dans ce notebook
import folium

Analysez le code suivant :
- ligne 1 : On crée une carte avec la fonction `folium.Map` avec une localisation de départ et un niveau de zoom 
- ligne 2 : on place un marqueur en précisant la localisation et un message en pop-up
- ligne 3 : on place un disque en précisant la localisation, le rayon en m, la couleur
- ligne 4 : la carte est sauvegardée

In [ ]:
c= folium.Map(location=[48.8,2.3],zoom_start=12)#zoom : 5=continent; 15=ville ; 10 = region
folium.Marker([48.85,  2.34],popup="Mon endroit").add_to(c)
folium.Circle([48.8,  2.6],popup="ma bulle",radius = 1000,color='crimson',fill=True,fill_color='crimson').add_to(c)
c.save('./maCarte.html')

In [ ]:
#pour ouvrir un onglet directement dans le navigateur
import webbrowser
webbrowser.open('./maCarte.html')

## EXERCICE 3 : Réaliser une carte du monde localisant les derniers gros tremblements de terre. Ceux-ci sont représentés par des disques dont le rayon indique la magnétude

(on peut aussi améliorer en jouant sur les couleurs etc ... )